In [5]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
crime_full = pd.read_csv('data/crime_data_2020_zipcode_final.csv', index_col=0)
crime_full.head(1)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year,ZIP
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978,2020.0,90037


In [9]:
crime_full['Crime Type'] = None

tqdm.pandas()
for index, row in tqdm(crime_full.iterrows(), total=len(crime_full), desc="Processing rows"):
    type = row['Crm Cd Desc']

    if 'RAPE' in type or 'ORAL' in type or 'SEXUAL' in type or 'LEWD' in type or 'SEX,UNLAWFUL' in type or 'CHILD PORNOGRAPHY' in type:
        crime_full.at[index, 'Crime Type'] = 'Sex-Related'

    elif 'ATTEMPTED' in type or 'ATTEMPT' in type:
        crime_full.at[index, 'Crime Type'] = 'Attempted Crime'
        
    elif 'ATTEMPTED' not in type:
        if 'ASSAULT' in type or 'Battery' in type:
            crime_full.at[index, 'Crime Type'] = 'Assault'
        
        elif 'THEFT' in type or 'STOLEN' in type or 'PURSE SNATCHING' in type or 'TILL TAP' in type or 'DRUNK ROLL' in type:
            crime_full.at[index, 'Crime Type'] = 'Larceny'
        
        elif 'ROBBERY' in type:
            crime_full.at[index, 'Crime Type'] = 'Robbery'
        
        elif 'VANDALISM' in type:
            crime_full.at[index, 'Crime Type'] = 'Vandalism'
        
        elif 'BURGLARY' in type:
            crime_full.at[index, 'Crime Type'] = 'Burglary'
        
        elif 'CHILD' in type or 'CHLD' in type:
            crime_full.at[index, 'Crime Type'] = 'Child Safety'

        elif 'THROWING OBJECT AT MOVING' in type or 'DISTURBING THE PEACE' in type or 'DISRUPT SCHOOL' in type or 'ARSON' in type:
            crime_full.at[index, 'Crime Type'] = 'Public Safety'
        
        else:
            crime_full.at[index, 'Crime Type'] = 'Others'

Processing rows: 100%|██████████| 20/20 [00:00<00:00, 10272.60it/s]


In [10]:
def get_season(in_file):
    for index, row in in_file.iterrows():
        date_str = row['DATE OCC']
        date_obj = datetime.strptime(date_str, "%m/%d/%Y %I:%M:%S %p")
        month = date_obj.month
        year = date_obj.year
        if month >= 1 and month <= 3:
            quarter = 'Q1'
        elif month >= 4 and month <= 6:
            quarter = 'Q2'
        elif month >= 7 and month <= 9:
            quarter = 'Q3'
        elif month >= 10:
            quarter = 'Q4'
        
        in_file.at[index, 'Season'] = str(year) + ' ' + quarter

get_season(crime_full)

In [12]:
crime_full = crime_full[crime_full['Season'] != '2023 Q4']
crime_full = crime_full.groupby(['ZIP', 'Crime Type', 'Season']).size().reset_index(name='Crime_Count')

crime_full.to_csv('crime_2020_viz.csv')
crime_full.head(3)

,ZIP,Crime Type,Season,Crime_Count
0,90012,Larceny,2020 Q1,1
1,90012,Others,2020 Q1,1
2,90013,Assault,2020 Q1,1
